In [3]:
import os
import glob
import pickle5 as pickle
import dlib
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [4]:
face_detector = dlib.get_frontal_face_detector()
points_detector = dlib.shape_predictor("data/recursos/shape_predictor_68_face_landmarks.dat")
face_recognition = dlib.face_recognition_model_v1("data/recursos/dlib_face_recognition_resnet_model_v1.dat")

indexes = np.load("data/recursos/yalefaces_indexes.picke", allow_pickle=True)
facial_descriptors = np.load("data/recursos/yalefaces_descriptors.npy")

In [5]:
def mode(predictions) -> str:
    idx = {}

    mx_value, ans = 0, None

    for prediction in predictions:
        idx.get
        if prediction not in idx:
            idx[prediction] = 1
        else:
            idx[prediction] += 1

    for key in idx:
        if idx[key] > mx_value:
            mx_value = idx[key]
            ans = key

    return ans
    

def knn(test, threshold = 0.5, k = 1):
    dists = []

    for i, sample in enumerate(facial_descriptors):
        dist = np.linalg.norm(test - sample)
        
        dists.append((dist, indexes[i]))


    dists.sort()
    dists = dists[0:k]

    predictions = [dist[1] for dist in dists if dist[0] <= threshold]

    return mode(predictions)

In [6]:
def get_person_name_by_img_name(img_name: str) -> str:
    name = img_name.split("/")[-1].split(".")[0]
    return name

In [13]:
correct_predictions = 0
total_predictions = 0

for img_file in glob.glob(os.path.join("data/yalefaces/teste", "*.gif")):
    img = Image.open(img_file).convert("RGB")
    img = np.array(img, dtype=np.uint8)
    faces = face_detector(img, 2)
    
    total_predictions += 1

    for face in faces:
        face_points = points_detector(img, face)
        face_descriptor = face_recognition.compute_face_descriptor(img, face_points)
        img_face_descriptors = np.asarray([df for df in face_descriptor], dtype=np.float64)

        prediction = knn(img_face_descriptors)
        y = get_person_name_by_img_name(img_file)

        if prediction == y:
            correct_predictions += 1

    #     x_min, y_max = face.tl_corner().x, face.tl_corner().y
    #     x_max, y_min = face.br_corner().x, face.br_corner().y

    #     cv.rectangle(img, (x_min, y_max), (x_max, y_min), (0,0,255), 1)
    #     cv.putText(img, prediction, (x_min, max(y_max - 10,0)), cv.FONT_HERSHEY_COMPLEX_SMALL, 0.5, (0,0,255), 1, cv.LINE_AA)

    # plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    # plt.show()

accuracy = correct_predictions/total_predictions
print(f"Total Number of Predictions: {total_predictions}")
print(f"Total Number of Correct Predictions: {correct_predictions}")
print(f"Accuracy: {accuracy:.4f}")

Total Number of Predictions: 30
Total Number of Correct Predictions: 29
Accuracy: 0.9667
